In [1]:
import polars as pl

df = pl.read_csv("data/llama3_raw_03.06.2024.csv")
print(df.columns)

cid,premise,hypothesis,label,Generated Entailment Hypothesis,Correct?,Adversarial?,Generated Neutral Hypothesis,Correct?_duplicated_0,Adversarial,Generated Contradiction Hypothesis,Correct?_duplicated_1,Adversarial_duplicated_0
i64,str,str,str,str,bool,bool,str,bool,bool,str,bool,bool
184052,"""Gangs of New York . The screen…","""Kenneth Lonergan is a writer o…","""ENTAILMENT""","""Kenneth Lonergan wrote the scr…",false,false,"""Kenneth Lonergan's birthdate i…",true,false,"""The screenplay of Gangs of New…",true,false
217178,"""Pakistan Movement . The Pakist…","""The Pakistan Movement was the …","""NEUTRAL""",""" The Pakistan Movement was a s…",true,false,"""The Pakistan Movement was infl…",true,false,"""The Pakistan Movement was a mo…",true,false
169984,"""The Brat Pack is a nickname gi…","""The Brat Pack is a nickname gi…","""CONTRADICTION""","""The Brat Pack was a group of a…",true,false,"""The Brat Pack was a formal org…",false,false,"""The Brat Pack was a group of a…",true,false
120315,"""Bret Easton Ellis . Ellis also…","""Bret Easton Ellis barely wrote…","""CONTRADICTION""","""Bret Easton Ellis was involved…",true,false,"""The Canyons is a film that bel…",false,false,"""The screenplay for The Canyons…",true,false
145735,"""New York Knicks . The Knicks c…","""The New York Knicks are in the…","""ENTAILMENT""","""The New York Knicks are a team…",true,false,"""The New York Knicks have won a…",true,false,"""The New York Knicks are a team…",true,false
